# Validate custom dataset

## Create dataset

In [ ]:
import fiftyone as fo

# Create the dataset
name = "mercedes-dataset"
dataset_dir = "../datasets/mercedes"  # Change with your path

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.COCODetectionDataset, # Change with your type
    name=name,
)

# !!! Color setting for labels, pls choose "Color annotations by value" oder instance
session = fo.launch_app(dataset)

In [ ]:
import fiftyone as fo

print(fo.list_datasets())
fo.delete_dataset("mercedes-dataset")

## Inference custom

In [ ]:
from mmdet.apis import DetInferencer
import fiftyone as fo

name = "mercedes-dataset"

# Load the dataset
dataset = fo.load_dataset(
    name=name,
)

model_config = "../models/mercedes/rtmdet-ins-233/mercedes_rtmdet-ins_l_8xb32-300e_coco.py"  # Change with your config
model_path = "../models/mercedes/rtmdet-ins-233/epoch_224.pth"  # Change with your model path

inferencer = DetInferencer(model=model_config, weights=model_path)

In [ ]:
print(dataset)

In [ ]:
for sample in dataset:
    results = inferencer(inputs=sample.filepath)

    print(results["predictions"][0].keys())

In [ ]:
results["predictions"][0]["bboxes"]

### load dataset

In [1]:
import fiftyone as fo

print(fo.list_datasets())
fo.delete_dataset("mercedes-dataset")

# Create the dataset
name = "mercedes-dataset"
dataset_dir = "../datasets/mercedes"  # Change with your path

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.COCODetectionDataset, # Change with your type
    name=name,
)

# !!! Color setting for labels, pls choose "Color annotations by value" oder instance
session = fo.launch_app(dataset)

/home/stan/miniconda3/envs/voxel51/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['coffe_1600', 'coffee', 'mercedes-dataset', 'quickstart-25']
 100% |███████████████████| 47/47 [12.4s elapsed, 0s remaining, 3.4 samples/s]      


### add custom bbox

In [2]:
from mmdet.apis import DetInferencer
import fiftyone as fo
import numpy as np

import tqdm

def convert_bbox_format_to_voxel51(bbox, img_w, img_h):

    x0, y0, x1, y1 = bbox

    # Normalize X and Y by width and height
    nx = x0 / img_w
    ny = y0 / img_h

    # Calculate width and height and normalize as well
    nw = (x1 - x0) / img_w
    nh = (y1 - y0) / img_h
    
    new_box = [nx, ny, nw, nh]

    return new_box

dataset.compute_metadata()

model_config = "../models/mercedes/rtmdet-ins-233/mercedes_rtmdet-ins_l_8xb32-300e_coco.py"  # Change with your config
model_path = "../models/mercedes/rtmdet-ins-233/epoch_224.pth"  # Change with your model path

inferencer = DetInferencer(model=model_config, weights=model_path)

pre_detections = []
classes = (
            'lc1','lc2','lc3','lc4','lc5','lc6','lc7',
            'lc8','lc9','lc10','lc11','lc12','lc13','lc14',
            'lc15','lc16','lc17','lc18','lc19','lc20','lc21'
)

for sample in tqdm.tqdm(dataset):
    results = inferencer(inputs=sample.filepath)

    labels = results["predictions"][0]["labels"]
    bboxes = results["predictions"][0]["bboxes"]

    for idx, bbox in enumerate(bboxes):

        bbox_voxel51 = convert_bbox_format_to_voxel51(bbox, sample.metadata.width, sample.metadata.height)

        detection = fo.Detection(
            label=classes[labels[idx]-1], bounding_box=bbox_voxel51
        )

        pre_detections.append(detection)

    sample["pre_detections"] = fo.Detections(detections=pre_detections)
    sample.save()

Loads checkpoint by local backend from path: ../models/mercedes/rtmdet-ins-233/epoch_224.pth
10/06 14:57:49 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/stan/miniconda3/envs/voxel51/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
  0%|          | 0/47 [00:00<?, ?it/s]

/home/stan/miniconda3/envs/voxel51/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/stan/miniconda3/envs/voxel51/lib/python3.10/site-packages/mmdet/models/layers/se_layer.py:158: FutureWarning:
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):

/home/stan/miniconda3/envs/voxel51/lib/python3.10/site-packages/mmdet/models/backbones/csp_darknet.py:118: 
FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` 
instead.
  with torch.cuda.amp.autocast(enabled=False):

...

 91%|█████████▏| 43/47 [04:46<00:27,  6.87s/it]

 94%|█████████▎| 44/47 [04:53<00:21,  7.10s/it]

 96%|█████████▌| 45/47 [05:01<00:14,  7.20s/it]

 98%|█████████▊| 46/47 [05:08<00:07,  7.25s/it]

100%|██████████| 47/47 [05:15<00:00,  6.72s/it]


In [3]:
session.show()